# LEAF CLASSIFICATION PROJECT - SVC and feature extraction

**TEAM**  
Kaido  
Sander  
Egert

**Training models that can classify leafs**  
**LINK:** https://www.kaggle.com/competitions/leaf-classification/data

Installing required packages

In [1]:
!pip install --upgrade pip
!pip install scikit-image

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [2]:
# imports
import os
import cv2
import pandas as pd
import numpy as np
import glob
import matplotlib.image as mpimg
from PIL import Image, ImageOps
from matplotlib import rcParams
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold, GridSearchCV
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.neural_network import MLPClassifier
from sklearn.pipeline import make_pipeline
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from skimage.feature import hog, local_binary_pattern
from skimage.measure import label, regionprops
from skimage.morphology import skeletonize, remove_small_objects
from skimage.filters import threshold_otsu
from skimage import exposure
from scipy import ndimage as ndi
from skimage.measure import shannon_entropy
from skimage.transform import resize
import math
import warnings

warnings.filterwarnings("ignore", category=UserWarning)

# Reading in the data

In [4]:

train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [5]:
test.head()

,id,margin1,margin2,margin3,margin4,margin5,margin6,margin7,margin8,margin9,...,texture55,texture56,texture57,texture58,texture59,texture60,texture61,texture62,texture63,texture64
0,4,0.019531,0.009766,0.078125,0.011719,0.003906,0.015625,0.005859,0.0,0.005859,...,0.006836,0.000000,0.015625,0.000977,0.015625,0.0,0.0,0.000000,0.003906,0.053711
1,7,0.007812,0.005859,0.064453,0.009766,0.003906,0.013672,0.007812,0.0,0.033203,...,0.000000,0.000000,0.006836,0.001953,0.013672,0.0,0.0,0.000977,0.037109,0.044922
2,9,0.000000,0.000000,0.001953,0.021484,0.041016,0.000000,0.023438,0.0,0.011719,...,0.128910,0.000000,0.000977,0.000000,0.000000,0.0,0.0,0.015625,0.000000,0.000000
3,12,0.000000,0.000000,0.009766,0.011719,0.017578,0.000000,0.003906,0.0,0.003906,...,0.012695,0.015625,0.002930,0.036133,0.013672,0.0,0.0,0.089844,0.000000,0.008789
4,13,0.001953,0.000000,0.015625,0.009766,0.039062,0.000000,0.009766,0.0,0.005859,...,0.000000,0.042969,0.016602,0.010742,0.041016,0.0,0.0,0.007812,0.009766,0.007812


In [6]:
# Convert y to integers
# Creates encoder and converts strings to integers
le = LabelEncoder()
y = le.fit_transform(train.species)

# Make dict of labels (int->str)
labels = {i: label for i, label in enumerate(le.classes_)}
train = train.drop(['species'], axis = 1)
print(labels)

{0: 'Acer_Capillipes', 1: 'Acer_Circinatum', 2: 'Acer_Mono', 3: 'Acer_Opalus', 4: 'Acer_Palmatum', 5: 'Acer_Pictum', 6: 'Acer_Platanoids', 7: 'Acer_Rubrum', 8: 'Acer_Rufinerve', 9: 'Acer_Saccharinum', 10: 'Alnus_Cordata', 11: 'Alnus_Maximowiczii', 12: 'Alnus_Rubra', 13: 'Alnus_Sieboldiana', 14: 'Alnus_Viridis', 15: 'Arundinaria_Simonii', 16: 'Betula_Austrosinensis', 17: 'Betula_Pendula', 18: 'Callicarpa_Bodinieri', 19: 'Castanea_Sativa', 20: 'Celtis_Koraiensis', 21: 'Cercis_Siliquastrum', 22: 'Cornus_Chinensis', 23: 'Cornus_Controversa', 24: 'Cornus_Macrophylla', 25: 'Cotinus_Coggygria', 26: 'Crataegus_Monogyna', 27: 'Cytisus_Battandieri', 28: 'Eucalyptus_Glaucescens', 29: 'Eucalyptus_Neglecta', 30: 'Eucalyptus_Urnigera', 31: 'Fagus_Sylvatica', 32: 'Ginkgo_Biloba', 33: 'Ilex_Aquifolium', 34: 'Ilex_Cornuta', 35: 'Liquidambar_Styraciflua', 36: 'Liriodendron_Tulipifera', 37: 'Lithocarpus_Cleistocarpus', 38: 'Lithocarpus_Edulis', 39: 'Magnolia_Heptapeta', 40: 'Magnolia_Salicifolia', 41: 'M

# Training on the given image features

In [7]:
X_train, X_test, y_train, y_test = train_test_split(train, y, train_size = 0.8, test_size = 0.2, shuffle=True, random_state=1, stratify=y)

In [10]:
model = make_pipeline(
    StandardScaler(),
    SVC(kernel='rbf', C=10, gamma='scale', probability=True)
)
model.fit(X_train, y_train)

,steps,"[('standardscaler', ...), ('svc', ...)]"
,transform_input,None
,memory,None
,verbose,False
,copy,True
,with_mean,True
,with_std,True
,C,10
,kernel,'rbf'
,degree,3
,gamma,'scale'


In [11]:
# finding best parameters for the svc

param_grid = {
    'svc__C': [1, 5, 10, 20],
    'svc__gamma': ['scale', 0.1, 0.01, 0.001],
}

grid = GridSearchCV(
    estimator=model,
    param_grid=param_grid,
    cv=5,
    scoring="accuracy",
    n_jobs=-1
)

grid.fit(X_train, y_train)
print(grid.best_params_)
print(grid.best_score_)

{'svc__C': 5, 'svc__gamma': 'scale'}
0.9861157551150386


In [12]:
model = make_pipeline(
    StandardScaler(),
    SVC(kernel='rbf', C=5, gamma='scale', probability=True)
)
model.fit(X_train, y_train)

,steps,"[('standardscaler', ...), ('svc', ...)]"
,transform_input,None
,memory,None
,verbose,False
,copy,True
,with_mean,True
,with_std,True
,C,5
,kernel,'rbf'
,degree,3
,gamma,'scale'


In [13]:
kf = StratifiedKFold(5, shuffle=True, random_state=1)
scores = cross_val_score(model, train, y, cv=kf)

print("Mean accuracy:", scores.mean())

Mean accuracy: 0.9868686868686869


In [14]:
probs = model.predict_proba(test)
species_names = le.classes_  
df_probs = pd.DataFrame(probs, columns=species_names)
df_probs.insert(0, "id", test['id'].values)
df_probs.head()


,id,Acer_Capillipes,Acer_Circinatum,Acer_Mono,Acer_Opalus,Acer_Palmatum,Acer_Pictum,Acer_Platanoids,Acer_Rubrum,Acer_Rufinerve,...,Salix_Fragilis,Salix_Intergra,Sorbus_Aria,Tilia_Oliveri,Tilia_Platyphyllos,Tilia_Tomentosa,Ulmus_Bergmanniana,Viburnum_Tinus,Viburnum_x_Rhytidophylloides,Zelkova_Serrata
0,4,0.006362,0.007194,0.005044,0.006415,0.005247,0.009127,0.004526,0.004795,0.005757,...,0.007335,0.006340,0.008081,0.008176,0.007465,0.005993,0.005730,0.004891,0.012596,0.009234
1,7,0.005939,0.006150,0.011257,0.023090,0.003587,0.006604,0.013445,0.004704,0.006225,...,0.007724,0.009566,0.005694,0.005847,0.003663,0.015022,0.006922,0.017673,0.005272,0.010650
2,9,0.010036,0.081950,0.006467,0.009701,0.012768,0.009655,0.006050,0.009287,0.018740,...,0.009298,0.005611,0.008964,0.009000,0.006824,0.010639,0.011298,0.005466,0.006481,0.024145
3,12,0.007668,0.016817,0.006725,0.009869,0.005278,0.006163,0.009101,0.008209,0.012410,...,0.012216,0.006166,0.010075,0.008239,0.005931,0.015361,0.019296,0.006984,0.006709,0.019026
4,13,0.010532,0.012479,0.005293,0.007018,0.005643,0.006287,0.006651,0.008130,0.026072,...,0.012148,0.005439,0.013192,0.008943,0.011580,0.013382,0.021530,0.006326,0.007217,0.012050


# Feature extraction

In [15]:
LBP_P = 8
LBP_R = 1
HOG_PIXELS_PER_CELL = (8,8)
HOG_CELLS_PER_BLOCK = (2,2)
HOG_ORIENTATIONS = 9
HOG_FEATURES_PER_IMAGE = 64 

In [16]:
def read_gray(path):
    img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
    if img is None:
        raise ValueError(f"Can't read {path}")
    return img

In [17]:
def edge_features(img):
    # img: uint8 grayscale
    # Canny edges
    edges = cv2.Canny(img, 50, 150)
    edge_density = edges.mean()              # fraction of pixels that are edges
    # Connected components on edges
    lbl = label(edges > 0)
    props = regionprops(lbl)
    n_components = len(props)
    lengths = [p.area for p in props]        # area ~= length for 1px-thick edges
    mean_len = np.mean(lengths) if lengths else 0.0
    std_len = np.std(lengths) if lengths else 0.0
    return {
        "edge_density": float(edge_density),
        "edge_comp_count": int(n_components),
        "edge_mean_len": float(mean_len),
        "edge_std_len": float(std_len)
    }


In [18]:
def hu_moments(img):
    moments = cv2.moments(img.astype(np.uint8))
    hu = cv2.HuMoments(moments).flatten()
    # log transform for scale stability
    with np.errstate(divide='ignore'):
        hu_log = -np.sign(hu) * np.log10(np.abs(hu) + 1e-30)
    hu_log = np.nan_to_num(hu_log, nan=0.0, posinf=0.0, neginf=0.0)
    return {f"hu_{i+1}": float(hu_log[i]) for i in range(len(hu_log))}


In [19]:
def lbp_features(img):
    # Otsu threshold may be useful to reduce background noise; but leave raw for LBP
    lbp = local_binary_pattern(img, P=LBP_P, R=LBP_R, method="uniform")
    # histogram
    n_bins = int(lbp.max() + 1)
    hist, _ = np.histogram(lbp.ravel(), bins=n_bins, range=(0, n_bins), density=True)
    hist = hist.astype(float)
    # return named features
    return {f"lbp_{i}": float(hist[i]) for i in range(len(hist))}

In [20]:
def skeleton_features(img):
    # binarize then skeletonize
    th = threshold_otsu(img)
    bw = img > th
    bw = remove_small_objects(bw, min_size=20)
    skel = skeletonize(bw)
    skel_len = skel.sum()
    # branch points: count pixels with >2 neighbors
    kernel = np.ones((3,3), dtype=np.uint8)
    nb = ndi.convolve(skel.astype(np.uint8), kernel, mode='constant', cval=0)
    # neighbor count excludes center pixel; subtract 1 if center is 1
    nb_count = nb * skel
    branch_points = np.sum((nb_count - 1) > 2)  # >2 other neighbors
    return {"skeleton_len": float(skel_len), "skeleton_branch_points": int(branch_points)}


In [21]:
def morph_features(img):
    # find largest contour to extract area/perimeter/solidity/extent/compactness
    th = threshold_otsu(img)
    bw = (img > th).astype(np.uint8) * 255
    contours, _ = cv2.findContours(bw, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    if not contours:
        return {"area":0,"perimeter":0,"aspect_ratio":0,"solidity":0,"extent":0,"compactness":0}
    # choose largest by area
    cnt = max(contours, key=cv2.contourArea)
    area = cv2.contourArea(cnt)
    perimeter = cv2.arcLength(cnt, True)
    x,y,w,h = cv2.boundingRect(cnt)
    aspect_ratio = float(w)/h if h>0 else 0
    # solidity: area / convex area
    hull = cv2.convexHull(cnt)
    hull_area = cv2.contourArea(hull) if hull is not None else 0
    solidity = float(area)/hull_area if hull_area>0 else 0
    rect_area = w*h
    extent = float(area)/rect_area if rect_area>0 else 0
    compactness = (perimeter**2 / area) if area>0 else 0
    return {"area":float(area), "perimeter":float(perimeter), "aspect_ratio":float(aspect_ratio),
            "solidity":float(solidity), "extent":float(extent), "compactness":float(compactness)}


In [22]:
def entropy_features(img):
    ent = float(shannon_entropy(img))
    # FFT band energies
    f = np.fft.fft2(img.astype(float))
    fshift = np.fft.fftshift(f)
    mag = np.abs(fshift)
    # radial bins
    h,w = img.shape
    cy, cx = h//2, w//2
    Y, X = np.ogrid[:h, :w]
    r = np.sqrt((X-cx)**2 + (Y-cy)**2)
    maxr = np.hypot(cx, cy)
    bands = [0.0, 0.1, 0.25, 0.5, 1.0]  # normalized radii
    energies = []
    for i in range(len(bands)-1):
        lo = bands[i]*maxr
        hi = bands[i+1]*maxr
        mask = (r >= lo) & (r < hi)
        energies.append(np.sum(mag[mask]) / (np.sum(mask)+1e-12))
    energies = [float(e) for e in energies]
    return {"entropy": ent, "fft_band0": energies[0], "fft_band1": energies[1], "fft_band2": energies[2], "fft_band3": energies[3]}


In [23]:
def hog_compact(img, out_size=HOG_FEATURES_PER_IMAGE):
    # compute HOG descriptor (skimage)
    # normalize/rescale
    img_res = resize(img, (128,128), anti_aliasing=True)
    hog_desc = hog(img_res, orientations=HOG_ORIENTATIONS,
                   pixels_per_cell=HOG_PIXELS_PER_CELL,
                   cells_per_block=HOG_CELLS_PER_BLOCK,
                   block_norm='L2-Hys', feature_vector=True)
    # pool/reshape to fixed length (simple downsample/average)
    if len(hog_desc) == out_size:
        return {f"hog_{i}": float(hog_desc[i]) for i in range(out_size)}
    # downsample by reshaping into bins
    bins = np.array_split(hog_desc, out_size)
    pooled = np.array([np.mean(b) for b in bins])
    return {f"hog_{i}": float(pooled[i]) for i in range(len(pooled))}

In [24]:
def extract_all_features(path):
    img = read_gray(path)
    feats = {}
    feats.update(edge_features(img))
    feats.update(hu_moments(img))
    feats.update(lbp_features(img))
    feats.update(skeleton_features(img))
    feats.update(morph_features(img))
    feats.update(entropy_features(img))
    feats.update(hog_compact(img))
    return feats

In [25]:
image_folder = "images"
records = []

for fname in sorted(os.listdir(image_folder)):
    if not fname.lower().endswith(('.png','.jpg','.jpeg','.tif','.bmp')):
        continue
    path = os.path.join(image_folder, fname)
    try:
        feats = extract_all_features(path)
    except Exception as e:
        print("Error", fname, e)
        continue
    feats['id'] = int(os.path.splitext(fname)[0])
    records.append(feats)
df_new = pd.DataFrame(records)
df_new = df_new.set_index('id')
df_new = df_new.sort_index()
# Now join with existing features (df_features indexed by same id)
# df_features = df_features.set_index('id')  # if not already


In [26]:
train = train.set_index('id')

test = test.set_index('id')

train_combined = train.join(df_new, how="left")
test_combined  = test.join(df_new, how="left")

print(train_combined.shape)
print(train_combined.head())
# Save


(990, 290)
     margin1   margin2   margin3   margin4   margin5   margin6   margin7  \
id                                                                         
1   0.007812  0.023438  0.023438  0.003906  0.011719  0.009766  0.027344   
2   0.005859  0.000000  0.031250  0.015625  0.025391  0.001953  0.019531   
3   0.005859  0.009766  0.019531  0.007812  0.003906  0.005859  0.068359   
5   0.000000  0.003906  0.023438  0.005859  0.021484  0.019531  0.023438   
6   0.005859  0.003906  0.048828  0.009766  0.013672  0.015625  0.005859   

    margin8   margin9  margin10  ...    hog_54    hog_55    hog_56    hog_57  \
id                               ...                                           
1       0.0  0.001953  0.033203  ...  0.047776  0.035819  0.062007  0.080937   
2       0.0  0.000000  0.007812  ...  0.111191  0.020955  0.008689  0.056076   
3       0.0  0.000000  0.044922  ...  0.104286  0.015120  0.015876  0.065303   
5       0.0  0.013672  0.017578  ...  0.088377  0.019969

In [27]:
test_combined.head()

,margin1,margin2,margin3,margin4,margin5,margin6,margin7,margin8,margin9,margin10,...,hog_54,hog_55,hog_56,hog_57,hog_58,hog_59,hog_60,hog_61,hog_62,hog_63
id,,,,,,,,,,,,,,,,,,,,,
4,0.019531,0.009766,0.078125,0.011719,0.003906,0.015625,0.005859,0.0,0.005859,0.023438,...,0.066891,0.032566,0.061606,0.081177,0.090292,0.023530,0.027087,0.059389,0.100768,0.015933
7,0.007812,0.005859,0.064453,0.009766,0.003906,0.013672,0.007812,0.0,0.033203,0.023438,...,0.062907,0.000000,0.067826,0.099230,0.063293,0.000000,0.032568,0.060250,0.018432,0.000000
9,0.000000,0.000000,0.001953,0.021484,0.041016,0.000000,0.023438,0.0,0.011719,0.005859,...,0.079710,0.030529,0.087301,0.090235,0.086235,0.029059,0.061159,0.062992,0.095438,0.023612
12,0.000000,0.000000,0.009766,0.011719,0.017578,0.000000,0.003906,0.0,0.003906,0.001953,...,0.100834,0.000000,0.091307,0.120527,0.088125,0.000000,0.033339,0.081469,0.081383,0.000000
13,0.001953,0.000000,0.015625,0.009766,0.039062,0.000000,0.009766,0.0,0.005859,0.000000,...,0.066337,0.007938,0.068231,0.074895,0.040879,0.000000,0.007937,0.047249,0.011221,0.000000


In [42]:
# save the extracted feratures to new files 
train_combined['species'] = le.inverse_transform(y)
train_combined.head()
train_combined.to_csv("train_feat.csv", index=True)
test_combined.to_csv("test_feat.csv", index=True)

In [43]:
train_combined = train_combined.drop(['species'], axis = 1)

In [39]:
X_train, X_test, y_train, y_test = train_test_split(train_combined, y, train_size = 0.8, test_size = 0.2, shuffle=True, random_state=1, stratify=y)


In [40]:
model = make_pipeline(
    StandardScaler(),
    SVC(kernel='rbf', C=10, gamma='scale', probability=True)
)
model.fit(X_train, y_train)

,steps,"[('standardscaler', ...), ('svc', ...)]"
,transform_input,None
,memory,None
,verbose,False
,copy,True
,with_mean,True
,with_std,True
,C,10
,kernel,'rbf'
,degree,3
,gamma,'scale'


In [41]:
param_grid = {
    'svc__C': [0.1, 1, 5, 10, 20],
    'svc__gamma': ['scale', 0.1, 0.01, 0.001],
    'svc__kernel': ['rbf', 'poly', 'sigmoid'],
    'svc__degree': [2, 3],
    'svc__class_weight': [None, 'balanced']
}

grid = GridSearchCV(
    estimator=model,
    param_grid=param_grid,
    cv=5,
    scoring="accuracy",
    n_jobs=-1
)

grid.fit(X_train, y_train)
print(grid.best_params_)
print(grid.best_score_)

{'svc__C': 5, 'svc__class_weight': None, 'svc__degree': 2, 'svc__gamma': 0.001, 'svc__kernel': 'sigmoid'}
0.9962025316455696


# Best m

In [44]:
model = make_pipeline(
    StandardScaler(),
    SVC(kernel='sigmoid', C=5, gamma=0.001, class_weight = None, degree = 2, probability=True)
)
model.fit(train_combined, y)

,steps,"[('standardscaler', ...), ('svc', ...)]"
,transform_input,None
,memory,None
,verbose,False
,copy,True
,with_mean,True
,with_std,True
,C,5
,kernel,'sigmoid'
,degree,2
,gamma,0.001


In [45]:
from sklearn.model_selection import StratifiedKFold, cross_val_score

kf = StratifiedKFold(5, shuffle=True, random_state=1)
scores = cross_val_score(model, train_combined, y, cv=kf)

print("Mean accuracy:", scores.mean())

Mean accuracy: 0.9959595959595958


In [46]:
probs = model.predict_proba(test_combined)
species_names = le.classes_  
df_probs = pd.DataFrame(probs, columns=species_names)
df_probs.insert(0, "id", test_combined.index.values)
df_probs.head()

,id,Acer_Capillipes,Acer_Circinatum,Acer_Mono,Acer_Opalus,Acer_Palmatum,Acer_Pictum,Acer_Platanoids,Acer_Rubrum,Acer_Rufinerve,...,Salix_Fragilis,Salix_Intergra,Sorbus_Aria,Tilia_Oliveri,Tilia_Platyphyllos,Tilia_Tomentosa,Ulmus_Bergmanniana,Viburnum_Tinus,Viburnum_x_Rhytidophylloides,Zelkova_Serrata
0,4,0.007819,0.006190,0.002699,0.012573,0.002799,0.005276,0.003121,0.004503,0.007967,...,0.002002,0.005975,0.005342,0.006786,0.008158,0.007262,0.004591,0.004778,0.017948,0.006182
1,7,0.008402,0.004756,0.008946,0.021344,0.001810,0.003074,0.011922,0.003763,0.005259,...,0.005998,0.010015,0.004370,0.004092,0.002789,0.015282,0.007984,0.024817,0.005082,0.014352
2,9,0.010235,0.103604,0.006670,0.019757,0.013986,0.007577,0.013796,0.021649,0.031205,...,0.005753,0.003811,0.005780,0.009438,0.007017,0.014035,0.015918,0.005465,0.004163,0.021696
3,12,0.005132,0.024454,0.003110,0.011430,0.002893,0.002245,0.013361,0.010395,0.015105,...,0.007178,0.004196,0.009987,0.008362,0.005333,0.016592,0.046656,0.006909,0.004646,0.011890
4,13,0.010641,0.016377,0.004355,0.012859,0.003893,0.002776,0.008419,0.010239,0.022289,...,0.007770,0.004169,0.017843,0.009700,0.015115,0.016834,0.030301,0.009254,0.006387,0.009107


In [47]:
df_probs.to_csv("species_probabilities.csv", index=False)

# Trying calibration on the model to be more accurate

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(train_combined, y, train_size = 0.8, test_size = 0.2, shuffle=True, random_state=1, stratify=y)
X_test2, X_val, y_test2, y_val = train_test_split(X_test, y_test, train_size = 0.5, test_size = 0.5, shuffle=True, random_state=1, stratify=y_test)

In [ ]:
model = make_pipeline(
    StandardScaler(),
    SVC(kernel='sigmoid', C=5, gamma=0.001, class_weight = None, degree = 2, probability=True)
)
model.fit(X_train, y_train)

In [ ]:
from sklearn.calibration import CalibratedClassifierCV
calibrated = CalibratedClassifierCV(model, cv='prefit', method='isotonic')
calibrated.fit(X_val, y_val)
y_pred = calibrated.predict(X_test2)
print("Test accuracy:", accuracy_score(y_test2, y_pred))

In [ ]:
from sklearn.calibration import CalibratedClassifierCV
calibrated = CalibratedClassifierCV(model, cv=2, method='sigmoid')
calibrated.fit(X_test, y_test)


In [ ]:
probs = model.predict_proba(test_combined)
species_names = le.classes_  
df_probs = pd.DataFrame(probs, columns=species_names)
df_probs.insert(0, "id", test_combined.index.values)
df_probs.head()

In [ ]:
df_probs.to_csv("species_probabilities.csv", index=False)